# 🏭 Laboratorio Modbus TCP - Interacción y Captura de Tramas

**Notebook interactivo** para aprender a interactuar con el servidor Modbus TCP, leer/escribir variables y capturar tramas de red.

## 📋 Contenido

1. **Instalación de dependencias**
2. **Conexión al servidor Modbus TCP**
3. **Lectura de Input Registers** (sensores)
4. **Lectura de Holding Registers** (setpoints)
5. **Escritura de Holding Registers**
6. **Captura de tramas Modbus con PyShark**
7. **Análisis y visualización de paquetes**
8. **Gráficos de datos en tiempo real**

---

## ⚙️ Requisitos Previos

- Servidor Modbus TCP ejecutándose (`./start.sh`)
- Python 3.8+
- Permisos para captura de paquetes (sudo/tshark)

## 1️⃣ Instalación de Dependencias

Instalamos las librerías necesarias para interactuar con Modbus y capturar tramas de red.

In [ ]:
# Instalación de paquetes necesarios
!pip install -q pymodbus pyshark pandas matplotlib seaborn

print("✅ Dependencias instaladas:")
print("  - pymodbus: Cliente Modbus TCP")
print("  - pyshark: Captura y análisis de paquetes de red")
print("  - pandas: Manipulación de datos")
print("  - matplotlib/seaborn: Visualización")

## 2️⃣ Importar Librerías

Importamos todas las librerías necesarias para el análisis.

In [ ]:
import pyshark
from pymodbus.client import ModbusTcpClient
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
import warnings

warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Librerías importadas correctamente")

## 3️⃣ Conexión al Servidor Modbus TCP

Nos conectamos al servidor Modbus TCP que simula el PLC industrial.

In [ ]:
# Configuración del servidor Modbus TCP
MODBUS_HOST = "172.25.0.10"  # IP del contenedor modbus-server
MODBUS_PORT = 502

# Crear cliente Modbus TCP
client = ModbusTcpClient(MODBUS_HOST, port=MODBUS_PORT)

# Conectar al servidor
if client.connect():
    print(f"✅ Conectado al servidor Modbus TCP en {MODBUS_HOST}:{MODBUS_PORT}")
else:
    print(f"❌ Error: No se pudo conectar a {MODBUS_HOST}:{MODBUS_PORT}")
    print("   Asegúrate de que el servidor esté ejecutándose: ./start.sh")

## 4️⃣ Lectura de Input Registers (Sensores)

Los **Input Registers** son de solo lectura y contienen valores de sensores.

In [ ]:
# Leer Input Registers (FC4 - Read Input Registers)
# Los valores están multiplicados por 100

print("📊 LECTURA DE INPUT REGISTERS (SENSORES)\n" + "="*50)

# Leer 4 input registers comenzando desde la dirección 0
result = client.read_input_registers(address=0, count=4, slave=1)

if not result.isError():
    # Convertir valores (dividir por 100 para obtener el valor real)
    temp1 = result.registers[0] / 100.0
    temp2 = result.registers[1] / 100.0
    presion = result.registers[2] / 100.0
    nivel = result.registers[3] / 100.0
    
    # Mostrar resultados
    print(f"🌡️  IR0 (30001) - Temperatura 1: {temp1:.1f} °C")
    print(f"🌡️  IR1 (30002) - Temperatura 2: {temp2:.1f} °C")
    print(f"⚙️  IR2 (30003) - Presión:        {presion:.1f} bar")
    print(f"💧 IR3 (30004) - Nivel Tanque:  {nivel:.1f} %")
    
    # Guardar en diccionario
    sensores = {
        'Temperatura 1': temp1,
        'Temperatura 2': temp2,
        'Presión': presion,
        'Nivel': nivel
    }
else:
    print(f"❌ Error al leer Input Registers: {result}")

## 5️⃣ Lectura de Holding Registers (Setpoints)

Los **Holding Registers** se pueden leer y escribir. Contienen setpoints y configuraciones.

In [ ]:
# Leer Holding Registers (FC3 - Read Holding Registers)

print("\n⚙️  LECTURA DE HOLDING REGISTERS (SETPOINTS)\n" + "="*50)

# Leer 4 holding registers comenzando desde la dirección 0
result = client.read_holding_registers(address=0, count=4, slave=1)

if not result.isError():
    # Convertir valores
    sp_temp = result.registers[0] / 100.0
    sp_nivel = result.registers[1] / 100.0
    tiempo_ciclo = result.registers[2]
    modo = result.registers[3]
    
    # Mostrar resultados
    print(f"🎯 HR0 (40001) - Setpoint Temperatura: {sp_temp:.1f} °C")
    print(f"🎯 HR1 (40002) - Setpoint Nivel:       {sp_nivel:.1f} %")
    print(f"⏱️  HR2 (40003) - Tiempo Ciclo:        {tiempo_ciclo} ms")
    print(f"🔧 HR3 (40004) - Modo Operación:      {'AUTO' if modo == 1 else 'MANUAL'} ({modo})")
    
    # Guardar en diccionario
    setpoints = {
        'SP Temperatura': sp_temp,
        'SP Nivel': sp_nivel,
        'Tiempo Ciclo': tiempo_ciclo,
        'Modo': 'AUTO' if modo == 1 else 'MANUAL'
    }
else:
    print(f"❌ Error al leer Holding Registers: {result}")

## 6️⃣ Escritura de Holding Registers

Escribimos nuevos valores en los Holding Registers (FC6 - Write Single Register).

In [ ]:
# Escribir valores en Holding Registers (FC6 - Write Single Register)

print("\n✍️  ESCRITURA DE HOLDING REGISTERS\n" + "="*50)

# Cambiar Setpoint de Temperatura a 22.5°C
nuevo_sp_temp = 22.5
valor_modbus = int(nuevo_sp_temp * 100)  # Multiplicar por 100

result = client.write_register(address=0, value=valor_modbus, slave=1)
if not result.isError():
    print(f"✅ HR0: Setpoint Temperatura actualizado a {nuevo_sp_temp}°C")
else:
    print(f"❌ Error al escribir HR0: {result}")

# Cambiar Setpoint de Nivel a 60%
nuevo_sp_nivel = 60.0
valor_modbus = int(nuevo_sp_nivel * 100)

result = client.write_register(address=1, value=valor_modbus, slave=1)
if not result.isError():
    print(f"✅ HR1: Setpoint Nivel actualizado a {nuevo_sp_nivel}%")
else:
    print(f"❌ Error al escribir HR1: {result}")

# Cambiar Modo a MANUAL (0)
result = client.write_register(address=3, value=0, slave=1)
if not result.isError():
    print(f"✅ HR3: Modo cambiado a MANUAL")
else:
    print(f"❌ Error al escribir HR3: {result}")

# Verificar los cambios
print("\n🔍 Verificando cambios...")
result = client.read_holding_registers(address=0, count=4, slave=1)
if not result.isError():
    print(f"   SP Temp: {result.registers[0]/100:.1f}°C")
    print(f"   SP Nivel: {result.registers[1]/100:.1f}%")
    print(f"   Modo: {'AUTO' if result.registers[3]==1 else 'MANUAL'}")

## 7️⃣ Captura de Tramas Modbus con PyShark

Ahora capturaremos el tráfico de red para ver las tramas Modbus TCP en detalle.

**Nota**: Necesitas permisos para captura de paquetes (tshark instalado).

In [ ]:
# Captura de paquetes Modbus TCP en tiempo real
# Este código captura 10 paquetes Modbus y los analiza

print("🔍 CAPTURA DE TRAMAS MODBUS TCP\n" + "="*50)
print("⏳ Capturando paquetes... (esto puede tomar unos segundos)")

try:
    # Crear captura en vivo con filtro para Modbus (puerto 502)
    # Interface 'any' captura de todas las interfaces
    capture = pyshark.LiveCapture(
        interface='any',
        display_filter='tcp.port==502',
        use_json=True
    )
    
    # Capturar solo 10 paquetes
    packets = []
    for packet in capture.sniff_continuously(packet_count=10):
        packets.append(packet)
    
    print(f"✅ Capturados {len(packets)} paquetes Modbus TCP\n")
    
    # Analizar los primeros 3 paquetes
    for i, pkt in enumerate(packets[:3], 1):
        print(f"\n📦 PAQUETE {i}:")
        print(f"   Timestamp: {pkt.sniff_time}")
        
        if hasattr(pkt, 'ip'):
            print(f"   Origen:    {pkt.ip.src}:{pkt.tcp.srcport}")
            print(f"   Destino:   {pkt.ip.dst}:{pkt.tcp.dstport}")
        
        if hasattr(pkt, 'modbus'):
            print(f"   Función:   {pkt.modbus.func_code if hasattr(pkt.modbus, 'func_code') else 'N/A'}")
        
        print(f"   Tamaño:    {pkt.length} bytes")
    
except Exception as e:
    print(f"⚠️  No se pudo capturar en vivo: {e}")
    print("\nAlternativa: Captura manual con tcpdump/tshark")
    print("Ejecuta en otra terminal:")
    print("  sudo tcpdump -i any port 502 -w modbus_capture.pcap -c 50")

## 8️⃣ Análisis de Archivo PCAP

Si ya tienes un archivo de captura (.pcap), podemos analizarlo en detalle.

In [ ]:
# Análisis de archivo PCAP existente
import os

pcap_file = "modbus_capture.pcap"

if os.path.exists(pcap_file):
    print(f"📂 ANÁLISIS DE ARCHIVO: {pcap_file}\n" + "="*50)
    
    # Cargar archivo PCAP
    cap = pyshark.FileCapture(
        pcap_file,
        display_filter='modbus',
        use_json=True
    )
    
    # Analizar paquetes
    packet_data = []
    
    for pkt in cap:
        try:
            data = {
                'timestamp': str(pkt.sniff_time),
                'src_ip': pkt.ip.src if hasattr(pkt, 'ip') else 'N/A',
                'dst_ip': pkt.ip.dst if hasattr(pkt, 'ip') else 'N/A',
                'src_port': pkt.tcp.srcport if hasattr(pkt, 'tcp') else 'N/A',
                'dst_port': pkt.tcp.dstport if hasattr(pkt, 'tcp') else 'N/A',
                'length': int(pkt.length),
                'protocol': 'Modbus'
            }
            
            if hasattr(pkt, 'modbus'):
                if hasattr(pkt.modbus, 'func_code'):
                    data['function'] = int(pkt.modbus.func_code)
            
            packet_data.append(data)
        except:
            continue
    
    # Crear DataFrame
    df = pd.DataFrame(packet_data)
    
    print(f"✅ Paquetes analizados: {len(df)}\n")
    print(df.head(10))
    
    # Estadísticas
    print(f"\n📊 ESTADÍSTICAS:")
    print(f"   Total paquetes: {len(df)}")
    if 'function' in df.columns:
        print(f"\n   Distribución por función Modbus:")
        print(df['function'].value_counts())
    
else:
    print(f"⚠️  Archivo {pcap_file} no encontrado")
    print("\nPara crear un archivo de captura, ejecuta en terminal:")
    print(f"  sudo tcpdump -i any port 502 -w {pcap_file} -c 100")

## 9️⃣ Monitoreo en Tiempo Real con Gráficos

Leemos las variables Modbus continuamente y las graficamos en tiempo real.

In [ ]:
# Monitoreo en tiempo real - recolectar datos cada 2 segundos durante 30 segundos

print("📈 MONITOREO EN TIEMPO REAL\n" + "="*50)
print("⏳ Recolectando datos durante 30 segundos...")

# Listas para almacenar datos
timestamps = []
temp1_data = []
temp2_data = []
presion_data = []
nivel_data = []

# Recolectar datos durante 30 segundos (15 muestras cada 2 seg)
for i in range(15):
    try:
        # Leer Input Registers
        result = client.read_input_registers(address=0, count=4, slave=1)
        
        if not result.isError():
            timestamps.append(datetime.now())
            temp1_data.append(result.registers[0] / 100.0)
            temp2_data.append(result.registers[1] / 100.0)
            presion_data.append(result.registers[2] / 100.0)
            nivel_data.append(result.registers[3] / 100.0)
            
            print(f"  [{i+1}/15] ✓ Muestra capturada")
        
        time.sleep(2)
    except KeyboardInterrupt:
        print("\n⚠️  Monitoreo interrumpido")
        break
    except Exception as e:
        print(f"  ❌ Error: {e}")
        continue

print(f"\n✅ Recolectadas {len(timestamps)} muestras")

# Crear DataFrame
df_monitor = pd.DataFrame({
    'Timestamp': timestamps,
    'Temperatura 1 (°C)': temp1_data,
    'Temperatura 2 (°C)': temp2_data,
    'Presión (bar)': presion_data,
    'Nivel (%)': nivel_data
})

print("\n📊 Primeras 5 muestras:")
print(df_monitor.head())

## 🔟 Visualización de Datos

Creamos gráficos profesionales de los datos monitoreados.

In [ ]:
# Crear visualizaciones de los datos monitoreados

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('📊 Monitoreo Modbus TCP - Variables en Tiempo Real', fontsize=16, fontweight='bold')

# Gráfico 1: Temperaturas
axes[0, 0].plot(df_monitor['Timestamp'], df_monitor['Temperatura 1 (°C)'], 
                marker='o', label='Temperatura 1', color='#FF6B6B', linewidth=2)
axes[0, 0].plot(df_monitor['Timestamp'], df_monitor['Temperatura 2 (°C)'], 
                marker='s', label='Temperatura 2', color='#4ECDC4', linewidth=2)
axes[0, 0].set_title('🌡️ Temperaturas', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Tiempo')
axes[0, 0].set_ylabel('Temperatura (°C)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# Gráfico 2: Presión
axes[0, 1].plot(df_monitor['Timestamp'], df_monitor['Presión (bar)'], 
                marker='D', color='#95E1D3', linewidth=2)
axes[0, 1].fill_between(df_monitor['Timestamp'], df_monitor['Presión (bar)'], 
                         alpha=0.3, color='#95E1D3')
axes[0, 1].set_title('⚙️ Presión', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Tiempo')
axes[0, 1].set_ylabel('Presión (bar)')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=45)

# Gráfico 3: Nivel del Tanque
axes[1, 0].plot(df_monitor['Timestamp'], df_monitor['Nivel (%)'], 
                marker='^', color='#F38181', linewidth=2)
axes[1, 0].fill_between(df_monitor['Timestamp'], df_monitor['Nivel (%)'], 
                         alpha=0.3, color='#F38181')
axes[1, 0].set_title('💧 Nivel del Tanque', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Tiempo')
axes[1, 0].set_ylabel('Nivel (%)')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].tick_params(axis='x', rotation=45)

# Gráfico 4: Todas las variables normalizadas
axes[1, 1].plot(df_monitor['Timestamp'], df_monitor['Temperatura 1 (°C)']/50*100, 
                label='Temp 1', alpha=0.7)
axes[1, 1].plot(df_monitor['Timestamp'], df_monitor['Temperatura 2 (°C)']/50*100, 
                label='Temp 2', alpha=0.7)
axes[1, 1].plot(df_monitor['Timestamp'], df_monitor['Presión (bar)']/30*100, 
                label='Presión', alpha=0.7)
axes[1, 1].plot(df_monitor['Timestamp'], df_monitor['Nivel (%)'], 
                label='Nivel', alpha=0.7)
axes[1, 1].set_title('📈 Todas las Variables (Normalizadas)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Tiempo')
axes[1, 1].set_ylabel('Valor Normalizado (%)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n✅ Gráficos generados exitosamente")

## 1️⃣1️⃣ Exportación de Datos

Guardamos los datos recolectados en formato CSV para análisis posterior.

In [ ]:
# Exportar datos a CSV
output_file = 'modbus_data.csv'

df_monitor.to_csv(output_file, index=False)
print(f"✅ Datos exportados a: {output_file}")
print(f"   Total de registros: {len(df_monitor)}")

# Estadísticas descriptivas
print("\n📊 ESTADÍSTICAS DESCRIPTIVAS:\n")
print(df_monitor.describe())

# Guardar también en JSON
json_file = 'modbus_data.json'
df_monitor.to_json(json_file, orient='records', date_format='iso')
print(f"\n✅ Datos también exportados a: {json_file}")

## 1️⃣2️⃣ Cerrar Conexión

Finalmente, cerramos la conexión con el servidor Modbus.

In [ ]:
# Cerrar conexión Modbus
client.close()
print("✅ Conexión Modbus cerrada")

print("\n" + "="*50)
print("🎉 ¡Notebook completado exitosamente!")
print("="*50)
print("\n📁 Archivos generados:")
print("   - modbus_data.csv")
print("   - modbus_data.json")
print("\n📚 Para más información:")
print("   - README.md")
print("   - ACCESO_NODE_RED.md")